In [ ]:
import ee
import geemap
import os
import datetime
import gc
from multiprocessing import Pool

In [ ]:
service_account = 'promedio-a--de-nieve-6bbfb64d1@ee-ezequiellagos.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'ee-ezequiellagos-0e8769baf6cb.json')
ee.Initialize(credentials)

In [ ]:
# Calculo de días a generar
days_delta = 4
date_start_str = '2000_02_24'

date_start = datetime.datetime.strptime(date_start_str, '%Y_%m_%d')
date_now = datetime.datetime.now() - datetime.timedelta(days=days_delta)
days_amount =  (date_now - date_start).days

# Lista de fechas
list_of_days = []
for i in range(days_amount + 1):
    date_now = date_start + datetime.timedelta(days=i)
    date_now_str = date_now.strftime('%Y_%m_%d')
    list_of_days.append(date_now_str)
list_of_days.reverse()

In [ ]:
Map = geemap.Map(center=[-32.639,-70.8484], zoom=9)
Map.clear_controls()

streamflow = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers").filter(ee.Filter.eq('BAS_ID', 3975615)).style(color= 'B2B2B3')
aconcagua_cuenca = ee.FeatureCollection("users/observatorionieves/DGA/aconcagua_cuenca").style(color= '#FF0000', width=1, fillColor= '00000000')
subsubcuencas_ms = ee.FeatureCollection("users/observatorionieves/DGA/aconcagua_subsubcuencas_ms")

Map.add_basemap('SATELLITE')

colorizedVis = {
  'min': 0.0,
  'max': 100.0,
  'palette': ['#ffffff','#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#253494','#081d58']
}

Map.addLayer(streamflow,{},'Red hídrica') 
Map.addLayer(aconcagua_cuenca,{},'Cuenca Río Aconcagua')
Map.addLayer(subsubcuencas_ms.style(fillColor= '00000000',color= '#000000', width= 1),{},'Selección media-superior cuenca del Río Aconcagua')
# Map

In [ ]:
def generate_html_per_day(date):
  try:
    current_time = datetime.datetime.now()

    MODIScollection = ee.Image('MODIS/006/MOD10A1/' + date).select(['NDSI_Snow_Cover'],['Snow_Albedo_Daily_Tile_Class']).clip(subsubcuencas_ms)
    Map.addLayer(MODIScollection,colorizedVis,"NDSI promedio") 
    
    # Generar HTML
    production = False
    if production:
        download_dir_base = 'web/media/maps/modis/doy/terra'
    else:
        download_dir_base = 'Downloads/maps/modis/doy/terra'
    
    html_file_name = date + '.html'
    title_html = 'MODIS Terra DOY - '+ date
    output = f'{current_time}: MODIS Terra DOY {date}'

    download_dir = os.path.join(os.path.expanduser('~'), download_dir_base)
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    html_file = os.path.join(download_dir, html_file_name)

    Map.to_html(outfile=html_file, title=title_html, width='100%', height='570px')
    print(output)

    Map.remove_ee_layer('NDSI promedio')

    # Liberar memoria
    gc.collect()

  except Exception as e:
    print(e)

pool_obj = Pool()
pool_obj.map(generate_html_per_day, list_of_days)


In [ ]:
# MODIScollection_2 = ee.ImageCollection('MODIS/006/MOD10A1').select('NDSI_Snow_Cover')
# print(MODIScollection_2.aggregate_array('system:id').getInfo())